In [40]:
%env PROJECT_ROOT_PATH=/Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection
%env MODEL=faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8
%env PIPELINE_CONFIG_FILE=pipeline_faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.config
#%env MODEL=ssd_resnet50_v1_fpn_640x640_coco17_tpu-8

env: PROJECT_ROOT_PATH=/Users/grubio/Downloads/image/multiple-image-detection/models/research/object_detection
env: MODEL=faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8
env: PIPELINE_CONFIG_FILE=pipeline_faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8.config


In [2]:
from tensorflow.python.client import device_lib

device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 377935090597837789]

In [33]:
!wget --quiet http://download.tensorflow.org/models/object_detection/tf2/20200711/$MODEL.tar.gz && mkdir -p $PROJECT_ROOT_PATH/trainning/models && tar -xzf $MODEL.tar.gz && mv $MODEL $PROJECT_ROOT_PATH/trainning/models && rm $MODEL.tar.gz

In [43]:
!ln -s $PROJECT_ROOT_PATH ~/

ln: /Users/grubio//object_detection: File exists


In [35]:
!cd $PROJECT_ROOT_PATH/object_detection/ && python -m pip install .

zsh:cd:1: no such file or directory: faster_rcnn_inception_resnet_v2_640x640_coco17_tpu-8/object_detection/


In [48]:
!TF_FORCE_GPU_ALLOW_GROWTH=true python $PROJECT_ROOT_PATH/model_main_tf2.py —logtostderr --model_dir=$PROJECT_ROOT_PATH/object_detection/trainning/models --pipeline_config_path=$PROJECT_ROOT_PATH/trainning/$PIPELINE_CONFIG_FILE

2021-01-10 18:22:34.909919: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-01-10 18:22:34.910438: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
W0110 18:22:34.911156 4674338240 cross_device_ops.py:1321] There are non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
I0110 18:22:34.911329 4674338240 mirrored_strategy.py:350] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: None
I0110 18:22:34.915042 4674338240 config_util.py:552] Maybe overwriting train_steps:

In [46]:
# ! python /object_detection/object_detection/exporter_main_v2.py  --input_type image_tensor --pipeline_config_path /object_detection/object_detection/trainning/pipeline.config --trained_checkpoint_dir /object_detection/object_detection/trainning/ --output_directory /object_detection/object_detection/trainning/exported-model/


In [2]:
! aws s3 sync --delete /object_detection/object_detection/trainning  s3://detectssss/trainning/10-jan-02

delete: s3://detectssss/trainning/ckpt-18.data-00000-of-00001
delete: s3://detectssss/trainning/ckpt-19.data-00000-of-00001  
delete: s3://detectssss/trainning/ckpt-18.index                
delete: s3://detectssss/trainning/ckpt-20.index                
delete: s3://detectssss/trainning/ckpt-19.index                
upload: ../object_detection/object_detection/trainning/ckpt-26.index to s3://detectssss/trainning/ckpt-26.index
upload: ../object_detection/object_detection/trainning/checkpoint to s3://detectssss/trainning/checkpoint
delete: s3://detectssss/trainning/ckpt-20.data-00000-of-00001    
upload: ../object_detection/object_detection/trainning/ckpt-25.index to s3://detectssss/trainning/ckpt-25.index
upload: ../object_detection/object_detection/trainning/ckpt-27.index to s3://detectssss/trainning/ckpt-27.index
upload: ../object_detection/object_detection/trainning/ckpt-25.data-00000-of-00001 to s3://detectssss/trainning/ckpt-25.data-00000-of-00001
upload: ../object_detection/object